<h1> DattoRMM - User Activity - MongoDB Ingest <h1>

# Import Modules and Set up MongoDB Client

In [ ]:
import re
import pandas as pd
import datetime as dt
import json


# add current timestamp to filename for reference
current_time = (dt.datetime.utcnow().strftime('%Y_%m_%d_%H%M%S'))

# export folder will contain all csv exported DataFrames for Ticket Creation
export_folder = 'd:/exports/'

# import configparser for env secrets
from configparser import ConfigParser
config = ConfigParser()
config.read('d:/git/example_infrastructure_data_dev/config/env.ini')
import requests
from requests.structures import CaseInsensitiveDict

# MongoDB

In [ ]:
username = config['mongodb']['username']
password = config['mongodb']['password']
connection_ip = config['mongodb']['connection_ip']
database = 'seed_data'

# import bson for object encoding
import bson

# Provide the mongodb atlas url to connect python to mongodb using pymongo
#CONNECTION_STRING = f"mongodb://{username}:{password}@{connection_ip}/{database}"
CONNECTION_STRING = 'mongodb://localhost:27017'


# Create a connection using MongoClient. You can import MongoClient or use pymongo.MongoClient
from pymongo import MongoClient
client = MongoClient(CONNECTION_STRING)


db = client['datto_rmm']
collection = db['user_audit']


# Import and Condition User Access CSV

In [ ]:
audit_logs_source = open('.txt','r',encoding='utf-8')
file = audit_logs_source.read()

In [ ]:
full_line_prog = re.compile(r'"(\d{4}-\d{2}-\d{2}\s\d{2}:\d{2}:\d{2}\s\bUTC\b)","([\w\d]{1}\S*@[^\.]*\.[^\"\']{2,5})","(\d{,4}\.\d{,4}\.\d{,4}\.\d{,4})","([^:]+):([^"]+)","(.*)"')

In [ ]:
parse = list(full_line_prog.findall(file))
parse_dict_list = []

In [ ]:
for item in parse[1:]:
    item_dict = {}
    list_item = list(item)
    item_dict['date'] = list_item[0]
    item_dict['username'] = list_item[1]
    item_dict['ipAddress'] = list_item[2]
    item_dict['group'] = list_item[3]
    item_dict['groupAction'] = list_item[4]
    item_dict['details'] = list_item[5]

    parse_dict_list.append(item_dict)

In [ ]:
df = pd.DataFrame(parse_dict_list)

In [ ]:
df[df['details'].str.contains(r"\*Software Management") & df['details'].str.contains(r"Christian")]

In [ ]:
df[df['date'] == '2022-01-12 21:06:28 UTC']

In [ ]:
dt_list = list(df_ingest.dtypes[df_ingest.dtypes == 'datetime64[ns]'].index)

In [ ]:
df_ingest.fillna(pd.NA,inplace=True)

In [ ]:
entries = []
cols = enumerate(list(df_ingest.columns))

for index, row in df_ingest.iterrows():
    current_row = row.to_dict()
    new_entry = {}
    try:
        try:
            new_entry['onlineData'] = {'legend':['onlineStatus','lastSeen'],'data':[[current_row['online'],current_row['lastSeen']]]}
        except:
            print(f'Unable to create column: "onlineData"')
        try:
            if current_row['suspended'] == 1:
                new_entry['isSuspended'] = {'legend':['suspendedStatus','lastSeen'],'data':[[current_row['suspended'],current_row['lastSeen']]]}
        except:
            print(f'Unable to create column: "isSuspended"')
            print(current_row['suspended'])
        try:
            if current_row['deleted'] == 1:
                new_entry['isDeleted'] = {'legend':['deletedStatus','lastSeen'],'data':[[current_row['deleted'],current_row['lastSeen']]]}
        except:
            print(f'Unable to create column: "isDeleted"')
            print(current_row['deleted'])
        try:
            if current_row['rebootRequired'] == 1:
                new_entry['rebootRequired'] = {'legend':['rebootRequired','lastReboot'],'data':[[current_row['rebootRequired'],current_row['lastReboot']]]}
        except:
            print(f'Unable to create column: "rebootRequired"')
            print(current_row['rebootRequired'])
        for k,v in current_row.items():
            if pd.isna(v) == True:
                continue
            elif (k == 'online') | (k == 'lastSeen') | (k == 'suspended') | (k == 'rebootRequired') | (k == 'lastReboot') |(k == 'deleted'):
                continue
            else:
                new_entry[k] = v

        entries.append(new_entry)
    except:
        print(current_row)

In [ ]:


for entry in entries:
    try:
        result = collection.find_one({'uid':entry['uid']})
        print(f"_id found: {result['_id']}")
        match_id = result['_id']
        for k,v in entry.items():
            try:
                #print(k,v)
                if k in  ['onlineData','isSuspended','isDeleted','rebootRequired']:
                    try:
                        #print({f"{ str(k)  + '.legend'}": v['legend']})
                        #collection.insert_one({'_id':m_id}, {f"{str(k)}":v['legend']})
                        collection.update_one({'_id':match_id},{'$set': {f"{ str(k)  + '.legend'}" : v['legend'] }},upsert=False)
                        collection.update_one({'_id':match_id},{'$push': {f"{ str(k)  + '.data'}" : {"$each":  v['data'] }}},upsert=True)
                        #print(f"Successfully updated {k} for {entry['hostname']}")
                    except Exception as error:
                        print(error)
                else:
                    try:
                        collection.update_one({'_id':match_id}, {'$set':{f"{str(k)}":v}},upsert=True)
                        #print(f"Successfully inserted new {k} for {entries[0]['hostname']}")
                    except Exception as error:
                        #print(f"Unable to insert new {k} for {entries[0]['hostname']}")
                        print(error)
            except Exception as e:
                print(e)
                continue
    except Exception as e:
        try:
            print(f"_id not found: attempting new entry for {entry['hostname']} with uid: {entry['uid']}")
            collection.insert_one(entry)
        except:
            print(e)

print('*'*120)
print('End of Import!')